In [ ]:
!pip install -q -U \
  "bitsandbytes==0.46.0" \
  "transformers==4.41.2" \
  "peft==0.11.1" \
  "accelerate==0.31.0" \
  "datasets==2.19.2" \
  "trl==0.8.6" \
  "huggingface_hub" \
  "minijinja" \
  "triton==3.2.0"

In [ ]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, get_peft_model, PeftModel
from trl import SFTTrainer
from huggingface_hub import login
import os

In [ ]:
import random
import numpy as np
import torch

def fix_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
from huggingface_hub import login
import os

login()

In [ ]:
MODEL_ID_FINETUNE = "meta-llama/Meta-Llama-3-8B-Instruct"

NEW_MODEL_NAME = f"{MODEL_ID_FINETUNE.split('/')[-1]}-synthetic-qlora"
OUTPUT_DIR = f"./{NEW_MODEL_NAME}-results"

print(f"Modelo base para fine-tuning: {MODEL_ID_FINETUNE}")
print(f"Nome do novo adaptador: {NEW_MODEL_NAME}")

In [ ]:
import torch

# Configuração de Quantização
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",       # Tipo de quantização em NormalFloat 4
    bnb_4bit_compute_dtype=torch.bfloat16, # Dtype para cômputo (float16, bfloat16)
    bnb_4bit_use_double_quant=False, # Usar quantização dupla (economiza um pouco mais de memória)
)

In [ ]:
print(f"Carregando modelo base: {MODEL_ID_FINETUNE}")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID_FINETUNE,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
print("Modelo carregado.")


tokenizer = AutoTokenizer.from_pretrained(MODEL_ID_FINETUNE, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Tokenizer carregado.")
print(f"PAD token ID: {tokenizer.pad_token_id}, EOS token ID: {tokenizer.eos_token_id}")

In [ ]:
import json

# Lista dos arquivos JSON da base MMLU
MMLU_INPUT_FILES = [
    'mmlu_data_1.json',
    'mmlu_data_2.json',
    'mmlu_data_3.json'
]

# Nome do arquivo para salvar os resultados da avaliação MMLU
MMLU_OUTPUT_FILE = 'mmlu_evaluation_results.jsonl'

# Template do Prompt
#    Este template inclui o texto inicial e a estrutura para os exemplos "few-shot".
MMLU_PROMPT_TEMPLATE = """The following are multiple choice questions (with answers) about various subjects. Choose the single most likely answer.

--- BEGIN EXAMPLES ---

(INSTRUCTION: Please replace the placeholders below with 4 high-quality, diverse examples to guide the model.)

[FEW-SHOT EXAMPLE 1]
Question: Which of the following is a type of sedimentary rock?
Choices:
A. Granite
B. Marble
C. Sandstone
D. Slate
Answer: C

[FEW-SHOT EXAMPLE 2]
Question: What is the capital of Japan?
Choices:
A. Beijing
B. Seoul
C. Tokyo
D. Bangkok
Answer: C

[FEW-SHOT EXAMPLE 3]
Question: Solve for x: 2x + 3 = 7
Choices:
A. 1
B. 2
C. 3
D. 5
Answer: B

[FEW-SHOT EXAMPLE 4]
Question: Who wrote "Hamlet"?
Choices:
A. Charles Dickens
B. William Shakespeare
C. Leo Tolstoy
D. Mark Twain
Answer: B

--- END EXAMPLES ---

Now, solve the following question. Provide only the letter of the correct answer.

Question: {question}
Choices:
{choices}
Answer:"""

print("Configurações da avaliação MMLU definidas.")

In [ ]:
from tqdm import tqdm
import os

def evaluate_mmlu(model,
                  tokenizer,
                  input_files,
                  output_file,
                  prompt_template,
                  max_new_tokens=5):
    """
    Avalia o modelo em questões da base MMLU a partir de arquivos JSON locais.

    Args:
        model: O modelo a ser testado.
        tokenizer: O tokenizer correspondente.
        input_files (list): Lista de caminhos para os arquivos JSON de entrada.
        output_file (str): Arquivo onde os resultados serão salvos.
        prompt_template (str): O template do prompt com os exemplos few-shot.
        max_new_tokens (int): Número máximo de tokens para a resposta (5 é suficiente para "A", "B", etc.).
    """
    print(f"Iniciando avaliação MMLU...")

    all_questions = []
    # Abre cada um dos arquivos JSON e agrega todas as questões em uma única lista
    for file_path in input_files:
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                all_questions.extend(data)
                print(f" - Carregadas {len(data)} questões de '{file_path}'")
        except FileNotFoundError:
            print(f"AVISO: Arquivo '{file_path}' não encontrado. Pulando.")
        except json.JSONDecodeError:
            print(f"AVISO: Arquivo '{file_path}' não é um JSON válido. Pulando.")

    if not all_questions:
        print("Nenhuma questão foi carregada. Abortando a avaliação.")
        return

    print(f"Total de {len(all_questions)} questões para avaliar.")

    if os.path.exists(output_file):
        os.remove(output_file)
        print(f"Arquivo de log antigo '{output_file}' removido.")

    # Loop principal de avaliação
    for i, item in enumerate(tqdm(all_questions, desc="Avaliando MMLU")):
        try:
            question = item['question']
            choices = item['choices']
            correct_answer_index = item.get('answer', -1)

            # Formata as opções de múltipla escolha
            formatted_choices = "\n".join([f"{chr(65+j)}. {choice}" for j, choice in enumerate(choices)])


            final_prompt = prompt_template.format(
                question=question,
                choices=formatted_choices
            )

            inputs = tokenizer(final_prompt, return_tensors="pt").to(model.device)

            with torch.no_grad():
                outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)

            # Decodifica apenas a resposta gerada
            generated_answer = tokenizer.decode(outputs[0][len(inputs['input_ids'][0]):], skip_special_tokens=True).strip()

            result_log = {
                "index": i,
                "subject": item.get('subject', 'N/A'),
                "question": question,
                "choices": choices,
                "correct_answer": choices[correct_answer_index] if correct_answer_index != -1 else "N/A",
                "correct_answer_letter": chr(65 + correct_answer_index) if correct_answer_index != -1 else "N/A",
                "generated_answer": generated_answer
            }

            # Salva o resultado de forma robusta a cada iteração
            with open(output_file, 'a', encoding='utf-8') as f:
                f.write(json.dumps(result_log, ensure_ascii=False) + '\n')

        except Exception as e:
            print(f"\nErro ao processar a questão {i}. Erro: {e}")
            error_log = {"index": i, "error": str(e)}
            with open(output_file, 'a', encoding='utf-8') as f:
                f.write(json.dumps(error_log, ensure_ascii=False) + '\n')

    print(f"\n🎉 Avaliação MMLU concluída! Resultados salvos em '{output_file}'.")

In [ ]:
# Chama a função de avaliação MMLU, passando o modelo base já carregado
evaluate_mmlu(
    model=model,
    tokenizer=tokenizer,
    input_files=MMLU_INPUT_FILES,
    output_file=MMLU_OUTPUT_FILE,
    prompt_template=MMLU_PROMPT_TEMPLATE
)